In [1]:
#Importing the Libraries to use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
#loading the test_set
data_test = pd.read_csv(r'C:\Users\Sir\python lessons\Zindi Hackathon\Financial Inclusion Dataset\Test set\Test.csv')
data_test.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent


In [3]:
#loading the training data
df = pd.read_csv(r'C:\Users\Sir\python lessons\Zindi Hackathon\Financial Inclusion Dataset\Training set\Train.csv')
df.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [14]:
#Our target column is the bank_account which we'll have to convert to numerical categorical variable
Target_dummy = pd.get_dummies(df['bank_account'])

#we the remove the bank_account column as we have no use for it now
df.drop('bank_account',axis =1, inplace = True)

#we add our encoded columns to our training set.Mind you now we have two columns Yes and No column
df = df.join(Target_dummy)
df.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,No,Yes
0,Kenya,2018,uniqueid_1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,0,1
1,Kenya,2018,uniqueid_2,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,1,0
2,Kenya,2018,uniqueid_3,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,0,1
3,Kenya,2018,uniqueid_4,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,1,0
4,Kenya,2018,uniqueid_5,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,1,0


In [15]:
#As one column can do the job,
#We remove The 'No' column
df.drop('No',axis = 1,inplace = True)

#Remaining with Yes we rename the column to Bank_account_Yes for proper visualization
df['Bank_account_Yes'] = df['Yes']
df.drop('Yes',axis = 1,inplace = True)
df.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,Bank_account_Yes
0,Kenya,2018,uniqueid_1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,1
1,Kenya,2018,uniqueid_2,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,0
2,Kenya,2018,uniqueid_3,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,1
3,Kenya,2018,uniqueid_4,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,0
4,Kenya,2018,uniqueid_5,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,0


In [16]:
#As we have different datasets for the training and test sets, we would like to make sure our pre_processing steps
# are similar
#That's why we use neHotEncoder with ColumnTransformer to ensure we fit_transform the training set and Transform the test set
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#We group the Categorical variables first
Categorical = ['country','location_type','cellphone_access','gender_of_respondent','relationship_with_head',
               'marital_status','education_level','job_type']
one_hot_encoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
column_transformer = ColumnTransformer(
    transformers = [('cat',one_hot_encoder,Categorical)
                   ],
    remainder = 'passthrough'
)
# Fit the transformer on training data and transform both training and test data
train_encoded = column_transformer.fit_transform(df.drop('Bank_account_Yes',axis = 1))
test_encoded = column_transformer.transform(data_test)

# Convert encoded data back to DataFrames
train_encoded_df = pd.DataFrame(train_encoded, columns=column_transformer.get_feature_names_out())
test_encoded_df = pd.DataFrame(test_encoded, columns=column_transformer.get_feature_names_out())

train_encoded_df['Bank_account_Yes'] = df["Bank_account_Yes"]

print("Encoded Training DataFrame:")
print(train_encoded_df)

Encoded Training DataFrame:
      cat__country_Kenya cat__country_Rwanda cat__country_Tanzania  \
0                    1.0                 0.0                   0.0   
1                    1.0                 0.0                   0.0   
2                    1.0                 0.0                   0.0   
3                    1.0                 0.0                   0.0   
4                    1.0                 0.0                   0.0   
...                  ...                 ...                   ...   
23519                0.0                 0.0                   0.0   
23520                0.0                 0.0                   0.0   
23521                0.0                 0.0                   0.0   
23522                0.0                 0.0                   0.0   
23523                0.0                 0.0                   0.0   

      cat__country_Uganda cat__location_type_Rural cat__location_type_Urban  \
0                     0.0                      1.0  

C:\Users\Sir\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
df = train_encoded_df
data_test = test_encoded_df
df.head()

,cat__country_Kenya,cat__country_Rwanda,cat__country_Tanzania,cat__country_Uganda,cat__location_type_Rural,cat__location_type_Urban,cat__cellphone_access_No,cat__cellphone_access_Yes,cat__gender_of_respondent_Female,cat__gender_of_respondent_Male,...,cat__job_type_Informally employed,cat__job_type_No Income,cat__job_type_Other Income,cat__job_type_Remittance Dependent,cat__job_type_Self employed,remainder__year,remainder__uniqueid,remainder__household_size,remainder__age_of_respondent,Bank_account_Yes
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018,uniqueid_1,3,24,1
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2018,uniqueid_2,5,70,0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018,uniqueid_3,5,26,1
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2018,uniqueid_4,5,34,0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,2018,uniqueid_5,8,26,0


In [25]:
Features = df.drop('Bank_account_Yes',axis =1)
Features = Features.drop('remainder__uniqueid',axis =1)
Target = df['Bank_account_Yes']
Features.head()

,cat__country_Kenya,cat__country_Rwanda,cat__country_Tanzania,cat__country_Uganda,cat__location_type_Rural,cat__location_type_Urban,cat__cellphone_access_No,cat__cellphone_access_Yes,cat__gender_of_respondent_Female,cat__gender_of_respondent_Male,...,cat__job_type_Formally employed Private,cat__job_type_Government Dependent,cat__job_type_Informally employed,cat__job_type_No Income,cat__job_type_Other Income,cat__job_type_Remittance Dependent,cat__job_type_Self employed,remainder__year,remainder__household_size,remainder__age_of_respondent
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018,3,24
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2018,5,70
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018,5,26
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,5,34
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2018,8,26


In [21]:
Target.head()

0    1
1    0
2    1
3    0
4    0
Name: Bank_account_Yes, dtype: uint8

# Note

During the EDA process I learnt that the training dataset was imbalanced. It had less Samples of the positive class **Yes**.
Also, the dataset is sorted by country and year.

I will shuffle the dataset first to clear the sorting then use Stratified Cross_validation to make sure my model
generalizes well to all parts of the training dataset.

In [27]:
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,y_train,y_test = train_test_split(Features,Target,test_size = 0.1,random_state = 30)

In [30]:
skf = StratifiedKFold(n_splits = 5,shuffle =True, random_state = 30)
Random_Forest = RandomForestClassifier()

import time
from sklearn.model_selection import GridSearchCV
# Start timing
start_time = time.time()

param_grid = {
        'n_estimators': [10, 100,200 ,300,],
        'max_depth': [10,20,30],
        'min_samples_split': [2,5,10,20],
        'min_samples_leaf': [1,2,4],
        'max_features': ['sqrt'],
        'bootstrap':[True],
        'class_weight':['balanced']
}

grid_search = GridSearchCV(estimator = Random_Forest, param_grid = param_grid, 
                           cv = skf, scoring = 'recall', n_jobs =-1,verbose =1)

grid_search.fit(X_train,y_train)

# End timing
end_time = time.time()



# Calculate elapsed time
elapsed_time = end_time - start_time

print("Best Parameters: ", grid_search.best_params_)
print("Best Score:",grid_search.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters:  {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 20, 'n_estimators': 300}
Best Score: 0.7450027244830653


# Note

After Performing the Grid Search altogether with the stratified cross validation,
These came up as the best parameters for use:
**Fitting 5 folds for each of 144 candidates, totalling 720 fits**
**Best Parameters:  {'bootstrap': True, 'class_weight': 'balanced',**
**'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 20, 'n_estimators': 300}**

I wanted to optimize recall so I decided to use recall as my scoring metric.
The Best score was **0.7450027244830653**

This works for me,now let me try out other evaluation metrics to gather more insight on my model

In [31]:
print(f'GridSearch:\n{elapsed_time}')

GridSearch:
1407.0293519496918


In [37]:
start_time = time.time()

Random_Forest = RandomForestClassifier(bootstrap = True, class_weight = 'balanced', max_depth = 10, 
                                       max_features = 'sqrt', min_samples_leaf = 2,
                                       min_samples_split = 20, n_estimators = 300)

Random_Forest.fit(X_train,y_train)
y_pred_train = Random_Forest.predict(X_train)
y_pred_test = Random_Forest.predict(X_test)

from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score

cf_matrix = confusion_matrix(y_test,y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)


end_time = time.time()
elapsed_time = end_time - start_time
print(f'Random Forest Fit_Transform:\n{elapsed_time}')

print(f'Training set accuracy:\n{accuracy_score(y_train,y_pred_train)}')
print(f'Test set accuracy:\n{accuracy_score(y_test,y_pred_test)}')
print(f'Confusion Matrix:\n{cf_matrix}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Random Forest Fit_Transform:
9.328362464904785
Training set accuracy:
0.80969250389684
Test set accuracy:
0.8147046323841904
Confusion Matrix:
[[1637  357]
 [  79  280]]
Precision: 0.44
Recall: 0.78


# Random_Forest Model

The model had an accuracy of **80%** in the training set and generalized well in the test set with **81%** accuracy
Since I had optimized recall the confusion matrix showed my model has a reduced number of false positives.

In [39]:
#Now we predict our hackathon test set that we had initially converted.
#Note: the test set had no bank_account column.
bank_account = Random_Forest.predict(data_test.drop('remainder__uniqueid',axis =1))

# Convert the predictions to a Pandas Series
bank_account_series = pd.Series(bank_account, index=data_test.index, name='bank_account')


data_test = data_test.join(bank_account_series)

data_test.head()

,cat__country_Kenya,cat__country_Rwanda,cat__country_Tanzania,cat__country_Uganda,cat__location_type_Rural,cat__location_type_Urban,cat__cellphone_access_No,cat__cellphone_access_Yes,cat__gender_of_respondent_Female,cat__gender_of_respondent_Male,...,cat__job_type_Informally employed,cat__job_type_No Income,cat__job_type_Other Income,cat__job_type_Remittance Dependent,cat__job_type_Self employed,remainder__year,remainder__uniqueid,remainder__household_size,remainder__age_of_respondent,bank_account
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2018,uniqueid_6056,3,30,1
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2018,uniqueid_6060,7,51,1
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2018,uniqueid_6065,3,77,0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2018,uniqueid_6072,6,39,0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,2018,uniqueid_6073,3,16,0


In [48]:
import pandas as pd

# Step 1: Create a new column to hold the country name based on the one-hot encoded columns
conditions = [
    data_test['cat__country_Kenya'] == 1,
    data_test['cat__country_Rwanda'] == 1,
    data_test['cat__country_Tanzania'] == 1,
    data_test['cat__country_Uganda'] == 1
]
choices = ['Kenya', 'Rwanda', 'Tanzania', 'Uganda']

data_test['country'] = np.select(conditions, choices)

# Step 2: Combine the 'uniqueid' with the identified country
data_test['unique_id'] = data_test['remainder__uniqueid'] + ' x ' + data_test['country']

# Now, you can view the new DataFrame
Submission_df = data_test[['unique_id','bank_account']]

Submission_df.head()



,unique_id,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0


# Prediction

Above is our Final prediction in the Test set.We will now go ahead and save it to an excel_file format.

In [52]:
Submission_df.to_excel("Financial Inclusion Prediction.xlsx",sheet_name = "Predictions", index = False)